In [1]:
from pytrends.request import TrendReq
from addon import *
import pandas as pd
import time
import pycountry

In [2]:

pytrends = TrendReq(hl='en-US', tz=360)

## %
kw_list = ["covid symptoms"]
df = pd.DataFrame()

In [3]:
states = [ 'AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'IL', 'IN', 'KS', 'KY', 'LA', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

In [4]:
i = 0
while i < len(states):
    try:
        pytrends.build_payload(kw_list, cat=0, timeframe='2020-03-22 2020-10-09', geo="US-" + states[i], gprop='')
        
        state_df = pytrends.interest_over_time()
        state_df["State"] = states[i]
        
        df = pd.concat([df, state_df])
        time.sleep(3)

        print(states[i])
        i += 1

        time.sleep(3)
    except Exception as e:
        i -= 1
        time.sleep(5)
        print(e)


AL
AZ
AR
CA
CO
CT
DE
DC
FL
GA
IL
IN
KS
KY
LA
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OK
OR
PA
RI
SC
SD
TN
TX
VT
VA
WA
WV
WI
WY


In [5]:
df.to_csv("../../data/trends/USA_statewise.csv")